LangChain Expression Language, or LCEL, is a declarative way to easily compose chains together. LCEL was designed from day 1 to support putting prototypes in production, with no code changes, from the simplest “prompt + LLM” chain to the most complex chains

## Setup and Import Libraries

In [15]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [2]:
load_dotenv()

True

In [4]:
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [5]:
llm = ChatGroq(model="Gemma2-9b-It")

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023E91C6A4D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023E91E24690>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

## Messages

In [7]:
messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

In [9]:
result = llm.invoke(messages)
result

AIMessage(content='Here are a few ways to say "Hello, how are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most formal way to greet someone)\n\n**Informal:**\n\n* **Salut, ça va ?** (This is a common and friendly greeting)\n* **Coucou, comment vas-tu ?** (This is a more casual and familiar greeting)\n\n\nLet me know if you\'d like to explore other ways to greet someone in French!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 21, 'total_tokens': 129, 'completion_time': 0.196363636, 'prompt_time': 0.00211833, 'queue_time': 0.016983359000000003, 'total_time': 0.198481966}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8b644680-e1c4-4618-8a97-7b1e79bac16d-0', usage_metadata={'input_tokens': 21, 'output_tokens': 108, 'total_tokens': 129})

## Output Parsers

In [11]:
parser = StrOutputParser()

In [12]:
parser.invoke(result)

'Here are a few ways to say "Hello, how are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most formal way to greet someone)\n\n**Informal:**\n\n* **Salut, ça va ?** (This is a common and friendly greeting)\n* **Coucou, comment vas-tu ?** (This is a more casual and familiar greeting)\n\n\nLet me know if you\'d like to explore other ways to greet someone in French!\n'

## LangChain Expression Language (LCEL)

In [14]:
chain = llm | parser
chain.invoke(messages)

'Here are a couple of ways to say "Hello, how are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most formal way to say it.)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (This is more casual and used with friends or family.) \n\n\n\nLet me know if you\'d like other variations!\n'

## Prompt Templates

In [16]:
generic_template = "Translate the following into {language}"

In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", generic_template),
        ("user","{text}")
    ]
)

In [20]:
result = prompt.invoke(
    {
        "language":"French",
        "text":"Hello"
    }
)
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into French', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})])

In [21]:
result.to_messages()

[SystemMessage(content='Translate the following into French', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [22]:
chain = prompt | llm | parser
chain.invoke(
    {
        "language":"French",
        "text":"Hello"
    }
)

'Hello in French is **Bonjour**. \n'